In [17]:
%pip install mediapipe opencv-python pandas numpy scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def extract_pose_from_video(video_path, output_csv="video_pose.csv"):
    cap = cv2.VideoCapture(video_path)
    landmarks_list = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
            landmarks_list.append(landmarks)
    
    cap.release()
    df = pd.DataFrame(landmarks_list)
    df.to_csv(output_csv, index=False)
    print(f"✅ Pose data saved to {output_csv}")
    return df


In [20]:
pose_df = extract_pose_from_video("IMG_3072.MOV")

✅ Pose data saved to video_pose.csv


In [21]:
# Load your datasets
train_df = pd.read_csv("clean_smash_dataset.csv")   # known good form data
video_df = pose_df                       # pose extracted from your video

# Step 1: Align column counts (important!)
min_cols = min(train_df.shape[1], video_df.shape[1])
train_df = train_df.iloc[:, :min_cols]
video_df = video_df.iloc[:, :min_cols]

# Step 2: Compute mean pose vectors
train_mean = train_df.mean().values.reshape(1, -1)
video_mean = video_df.mean().values.reshape(1, -1)

# Step 3: Compute similarity
similarity = cosine_similarity(video_mean, train_mean)[0][0]

print(f"🏸 Similarity score to reference form: {similarity:.3f}")

# Interpret score
if similarity > 0.95:
    print("Excellent — very close to good smash form!")
elif similarity > 0.85:
    print("Good form overall, minor deviations.")
elif similarity > 0.75:
    print("Fair form — needs some correction.")
else:
    print("⚠️ Significant deviation from good form.")

🏸 Similarity score to reference form: 0.612
⚠️ Significant deviation from good form.
